# Decesion Tree from scratch

### Assumption to consider

1. This decesion tree only works well with multi-class classfication problems
2. Decesion tree build during training will always be binary tree
3. Works for numerical and categorical data (but not optimised for numeric data).

In [4]:
import numpy as np

## Entropy

$$ H(p) = -\sum_{i=0}^{c} p_c\log{(p_c)}$$

##### where $ \sum_{i=0}^{c} p_c = 1 $ #####

##### $p_c$ is the frequency of the every ith category #####

In [5]:
def entropy(y):
    '''
    Calculates impurities in the target value 

    Args:       
        y ndarray(m, )

    Returns:
        entropy (scalar)
    '''
    categories, cnt = np.unique(y, return_counts = True)
    p = cnt / len(y)

    entropy = -np.sum(p * np.log(p))
    return entropy

# test 
y_temp = np.array([1,1, 0, 1])
print(f"my entropy: {entropy(y_temp)}") # o/p 0.5623351446188083

my entropy: 0.5623351446188083


In [6]:
def split(X, feature, threshold):
    '''
    Returns indices of left split and right split. This is done based on feature and threshold given.
    
    Args:
        X (ndarray(m, n))
        feature (scalar)
        threshold (scalar)
        
    Returns:
        left_idxs (ndarray(m2))
        right_idxs (ndarray(m2,))
    '''
    left_idxs = np.argwhere(X[:, feature] <= threshold).flatten()
    right_idxs = np.argwhere(X[:, feature] > threshold).flatten()

    return left_idxs, right_idxs

# test
x_temp = np.array([
    [1, 0, 1],
    [1, 0, 0],
    [1, 0, 0]
])
split(x_temp, 2, 0) # o/p: (array([1, 2]), array([0]))

(array([1, 2]), array([0]))

## Information Gain

$$ \text{infomation gain} = H(Parent) - H_w(Children) $$

$$ \text{where  } H_w(Children) = w_{left}*H(left) + w_{right}*H(right)$$ 

#### here __H__ is entropy
#### $H_w$ is weighted entropy
#### $w_{left} = \frac{\text{No. of datapoints in left node}}{\text{Total no. of datapoints in parent node}}$

In [7]:
def information_gain(X, y, feature, threshold):
    '''
    Args:
        X (ndarray(m, n))
        y (ndarray(m,))
        feature (scalar)
        threshold (scalar)

    Returns:
        info_gain (scalar)
    '''
    # split data into left and right idices
    left_idxs, right_idxs = split(X, feature, threshold)
    
    # Calculate entropy of parent and children node

    w_left = len(left_idxs) / len(y)
    w_right = len(right_idxs) / len(y)
    entropy_left = entropy(y[left_idxs])
    entropy_right = entropy(y[right_idxs])
    
    entropy_parent = entropy(y)
    entropy_children = w_left * entropy_left + w_right * entropy_right
    
    # calculate information gain
    info_gain =  entropy_parent - entropy_children

    return info_gain

# test
temp_X = np.array([
    [1,0,1], 
    [1,1,0], 
    [0,1,0]
])
temp_y = np.array([1,0,0])

temp_node_indices = np.array([0,1,2])
temp_feature = [0,1,2]

for f in temp_feature:
    info_gain = information_gain(temp_X, temp_y, f, 0)
    print(f"feature: {f}, info_gain: {info_gain}")
    
## output 
# feature: 0, info_gain: 0.17441604792151594
# feature: 1, info_gain: 0.6365141682948128
# feature: 2, info_gain: 0.6365141682948128

feature: 0, info_gain: 0.17441604792151594
feature: 1, info_gain: 0.6365141682948128
feature: 2, info_gain: 0.6365141682948128


In [8]:
def best_split(X, y):
    '''
    Returns the feature and threshold that is hightest information w.r.t given datapoints

    Args:
        X (ndarray(m,n))
        y (ndarray(m,))
    Returns:
        best_feature (scalar): index of the feature
        best_threshold (scalar): one of the category or number from the feature
    '''
    
    n_features = X.shape[1]

    best_feature = -1
    best_threshold = -1
    best_info_gain = -1
    
    for feature in range(n_features):
        categories = np.unique(X[:, feature])
        
        for threshold in categories:
            info_gain = information_gain(X, y, feature, threshold)

            if best_info_gain < info_gain:
                best_info_gain = info_gain
                best_threshold = threshold
                best_feature = feature
                
    return best_feature, best_threshold

# test
temp_X = np.array([
    [1,0,1], 
    [1,1,0], 
    [0,1,0]
])

temp_y = np.array([1,0,0])
temp_node_indices = np.array([0,1,2])
temp_feature = [0,1,2]

temp_best_feature, temp_best_threshold = best_split(temp_X, temp_y)
print(f"best feature: {temp_best_feature}, best thresold: {temp_best_threshold}") 

## Output
#  best feature: 1, best thresold: 0

best feature: 1, best thresold: 0


In [9]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, predicted_value=None):
        '''
        feature (scalar): Index of feature. On basis of this feature the node should split data.
        threshold (scalar): Value of the feature on from which other datapoint is compared for spliting 
        predicted_value (scalar): If node is leaf node in decesion then it will return prediction value. Otherwise it will be null.
        left (Node)
        right (Node)
        '''

        self.feature = feature
        self.threshold = threshold
        self.predicted_value = predicted_value

        self.left = left
        self.right = right

    def is_leaf_node(self):
        return self.predicted_value != None

## Base case in Build Tree 
Return leaf node if:
1. current node depth is equal to max_depth.
2. all given examples' target value is same.
3. number of examples are less than nnumber of splits we make of a child node(i.e 2)


In [10]:
def build_tree(X, y, curr_depth ,max_depth):
        '''
        It creates decesion tree based on given training data

        Args:
            X (ndarray(m, n)): m examples and n features
            y (ndarray(m, )): target value of m examples
            max_depth (scalar)
        Return:
            root (Node): return parent node
        '''
        m = X.shape[0]
    
        # base case
        if curr_depth == max_depth or len(np.unique(y)) == 1 or m < 2:
            # find most common label
            unique_labels, cnt = np.unique(y, return_counts=True)
            max_cnt_idx = np.argmax(cnt)
            most_common_label =  unique_labels[max_cnt_idx]

            return Node(predicted_value=most_common_label)
            
        # select best feature and threshold 
        feature, threshold = best_split(X, y)
    
        # split data based on best feature and threshold
        left_idxs, right_idxs = split(X, y, feature, threshold)
    
        # call child node
        left_node = build_tree(X[left_idxs], y[left_idxs], curr_depth+1, max_depth)
        right_node = build_tree(X[right_idxs], y[right_idxs], curr_depth+1, max_depth)

        return Node(feature, threshold, left_node, right_node)

In [11]:
class DecisionTree:
    def __init__(self, max_depth=100):
        self.max_depth = 100
        self.root = None

    def fit(self, X, y):
        self.root = build_tree(X, y, curr_depth=0, max_depth=self.max_depth)

    
    def score(self, X, y):
        y_pred = self.predict(X)

        return np.sum(y_pred == y) / len(y)
        
    def predict(self, X):
        if not self.root:
            raise ValueError("The tree has not been trained yet.")
            
        ans = np.array([self._traverse_tree(x, self.root) for x in X])
        return ans

    def _traverse_tree(self, x, node):
        '''Predicts target value for single datapoint'''
        if node.is_leaf_node():
            return node.predicted_value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
            
        return self._traverse_tree(x, node.right)

### Test decesion tree works well

In [13]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from DecisionTree import DecisionTree

data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

acc = clf.score(X_test, y_test)
print(f"Accuracy: {acc: .3f}")

Accuracy:  0.912
